In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

## 0. DataFrame

In [ ]:
football_df = pd.read_csv('all_data_with_elo.csv', low_memory = False)
football_df

## 1. Descriptive Statistics 

**1.1 DataFrame Shape**

In [ ]:
# no. rows and no. cols
football_df.shape

In [ ]:
# feature names
print(football_df.columns.tolist())

**1.2 NaN Values**

In [ ]:
football_df.isnull().sum()

In [ ]:
# total elements in 
football_df.size

In [ ]:
# total number of NaN
football_df.size - football_df.count().sum()

In [ ]:
# total number of NaN rows
football_df.isnull().any(axis = 1).sum()

In [ ]:
# total number of NaN columns
football_df.isnull().any(axis = 0).sum()

## 2. Data Wrangling and Feature Transformation/Development

**2.1 NaN Handling**

`TODO`: drop NaN values along columns: {Date, Home Team, Away Team, FTR} <br>
`TODO`: identify betting odds w/ most available data

In [ ]:
nan_mask = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 
            'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'HomeTeamELO', 'AwayTeamELO']

In [ ]:
#football_df.FTR.replace('nan', np.nan, inplace=True)
nan_football_df = football_df.dropna(subset = nan_mask)
nan_football_df

In [ ]:
# resize shape
football_df.shape[0] - nan_football_df.shape[0]

**2.2 Feature Transformation** <br>
* One hot encode Division, Home and Away Teams
* Label encode Full Time Result (Win/Draw/Loss)
* $\phi(Date)$ $\Rightarrow$ one column for *year*, second column for *day of year*
* $\phi(x)$ feature transformation $\Rightarrow$ win/loss streak to date
* Betting odds $\Rightarrow$ average the home, away, and draw odds from the two odd sites

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
feats = nan_mask

In [ ]:
learning_df = nan_football_df.copy()[feats]
learning_df.reset_index(inplace=True, drop=True)

**2.2.1 Division and Home/Away Team Encoding**

In [ ]:
div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()

In [ ]:
onehot_div = div_encoder.fit_transform(learning_df.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(learning_df.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(learning_df.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

In [ ]:
learning_df = pd.concat([learning_df, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
learning_df.drop(columns = ['Div'], inplace = True)

In [ ]:
learning_df

**2.2.2 Date Transformation**

In [ ]:
learning_df['Year'] = pd.DatetimeIndex(learning_df.Date).year
learning_df['DayofYear'] = pd.DatetimeIndex(learning_df.Date).dayofyear
learning_df.drop(columns = ['Date'], inplace = True)

**2.2.3 Full Time Result Encoding**

In [ ]:
target_encoder = LabelEncoder()
learning_df['Result'] = target_encoder.fit_transform(learning_df.FTR) 
learning_df.drop(columns = ['FTR'], inplace = True)

**2.2.4 Win/Loss Streak Feature Creation** <br>
Important note about this feature: the win/loss streak is the teams *home* and *away* win streak, *not* its ***consecutive*** win/loss streak.

In [ ]:
# https://stackoverflow.com/questions/52976336/compute-winning-streak-with-pandas
# https://joshdevlin.com/blog/calculate-streaks-in-pandas/

In [ ]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])

In [ ]:
learning_df = compute_winstreak(learning_df)
learning_df.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)

**2.2.5 Last Match Result** <br>
Indicate the result from the last match played between both teams

In [ ]:
## TODO ##

**2.2.6 Team Wins to Date**

In [ ]:
## TODO ##

**2.2.7 Website Odds** <br>
The `betting odds` recorded by various betting websites offer insight into sentiment surrounding the outcome of a particular game. 

In [ ]:
## TODO ##

In [ ]:
betting_feats = ['B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']
betting_feats

In [ ]:
def compute_odds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]

In [ ]:
learning_df = compute_odds(learning_df, betting_feats)

**2.2.8 Elo Difference**

In [ ]:
# hometeamELO = learning_df.HomeTeamELO - learning_df.AwayTeamELO

In [ ]:
# learning_df.drop(columns = ['HomeTeamELO', 'AwayTeamELO'], inplace = True)
# learning_df['HomeTeamELO'] = hometeamELO

**2.2.8 Peek @ Learning DataFrame**

In [ ]:
learning_df

## 3. Preliminary Regression

* Establish a baseline Linear Regression model fit over the entire learning dataframe without special regard to *division* and *team*. 
* Train model over 18 seasons, and predict for the remaining 3 seasons (approximate 80-20 split)

In [ ]:
from sklearn.metrics import accuracy_score

**3.1 Train and Test Split**

In [ ]:
split = 0.80
no_seasons = 20

print('No. seasons to train over: ' + str(round(split*no_seasons)))

In [ ]:
X, y = learning_df.loc[:, learning_df.columns != 'Result'], learning_df[['Result']]

In [ ]:
X

In [ ]:
xTr, xTe = X[X.Year <= 2016], X[X.Year > 2016]
yTr, yTe = y.loc[xTr.index, :], y.loc[xTe.index, :]

**3.2 Normalization** <br>
Following our various feature transformations and development, we arrived to a sparse dataframe with the exception of a few features(*Year, DayofYear*). It will be important to *normalize* these features as they are in gross magnitudes compared to the remaining features. During model training, having dominating features (in scale relative to others) can be dangerous as the weight updates may mistakengly favor these larger-scale features because it will have the largest influence on the target output. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
xTr.loc[:, ['Year', 'DayofYear']] = minmax_scaler.fit_transform(xTr.loc[:, ['Year', 'DayofYear']])
xTe.loc[:, ['Year', 'DayofYear']] = minmax_scaler.transform(xTe.loc[:, ['Year', 'DayofYear']])

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
to_scale = ['HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

xTr.loc[:, to_scale] = std_scaler.fit_transform(xTr.loc[:, to_scale])
xTe.loc[:, to_scale] = std_scaler.transform(xTe.loc[:, to_scale])

In [ ]:
xTr

**3.3 HomeWins Baseline Model**

In [ ]:
baseline_preds = np.full((8628, 1), 2) #predicts home wins all the time
accuracy_score(yTe.Result.values, baseline_preds.ravel())

**3.4 Multinomial Logistic Regression**

**3.4.1 Full Model Fit**

In [ ]:
from sklearn.linear_model import LogisticRegression
linear_model = LogisticRegression(max_iter = 10000).fit(xTr, yTr.values.ravel())

In [ ]:
lr_preds = linear_model.predict(xTe)
accuracy_score(yTe.Result.values, lr_preds)

In [ ]:
accuracy_score(yTr.Result.values, linear_model.predict(xTr))

**3.4.2 Parameter Tuning**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

logistic_params = {'penalty': ['l1','l2','elasticnet'],
                   'C':[0.001,0.01,0.10,0.50,1.0]}

logistic_randsearch = RandomizedSearchCV(estimator=LogisticRegression(max_iter=10000),
                                         param_distributions=logistic_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         cv=5)

logistic_rand_results = logistic_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (logistic_rand_results.best_score_, logistic_rand_results.best_params_))

In [ ]:
rs_preds = logistic_rand_results.best_estimator_.predict(xTe)

In [ ]:
accuracy_score(yTe.Result.values, rs_preds)

**3.4.3 Team Fit Model**

In [ ]:
X_barcelona = X[X['HomeTeam 18'] == 1].loc[:, 'AwayTeam 0':]
y_barcelona = y.loc[X_barcelona.index, :]

In [ ]:
bxTr, bxTe = X_barcelona[X_barcelona.Year <= 2016], X_barcelona[X_barcelona.Year > 2018]
byTr, byTe = y_barcelona.loc[bxTr.index, :], y_barcelona.loc[bxTe.index, :]

In [ ]:
bminmax_scaler = MinMaxScaler()
bxTr.loc[:, ['Year', 'DayofYear']] = minmax_scaler.fit_transform(bxTr.loc[:, ['Year', 'DayofYear']])
bxTe.loc[:, ['Year', 'DayofYear']] = minmax_scaler.transform(bxTe.loc[:, ['Year', 'DayofYear']])

In [ ]:
barcelona_model = LogisticRegression(max_iter = 10000).fit(bxTr, byTr.values.ravel())

In [ ]:
barcelona_preds = barcelona_model.predict(bxTe)
accuracy_score(byTe.Result.values, barcelona_preds)

In [ ]:
fullModel_preds = linear_model.predict(xTe[xTe['HomeTeam 18'] == 1])
accuracy_score(byTe.Result.values, fullModel_preds)

**3.5 Ridge Classifier**

In [ ]:
from sklearn.linear_model import RidgeClassifier
ridge_model = RidgeClassifier().fit(xTr, yTr.values.ravel())

In [ ]:
ridge_preds = ridge_model.predict(xTe)
accuracy_score(yTe.Result.values, ridge_preds)